In [4]:
import ray
from ray import rllib, tune
import pandas as pd
import numpy as np
from datetime import datetime
import trading_gym
from trading_gym.registry.gaia.v13.env import GAIAPredictorsContinuousV13
from trading_gym.ray.walkforward import WalkForwardRunner, WalkForwardResults
%matplotlib inline
print(trading_gym.__package__, trading_gym.__version__)
print(ray.__package__, ray.__version__)

trading-gym 0.8.1
ray 0.7.3


In [2]:
from ray.rllib.models.tf.tf_modelv2 import TFModelV2
from ray.rllib.models import ModelCatalog

In [3]:
ray.init()

2019-08-24 23:31:21,723	INFO node.py:498 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-08-24_23-31-21_722734_101587/logs.
2019-08-24 23:31:22,043	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:16309 to respond...
2019-08-24 23:31:22,277	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:32833 to respond...
2019-08-24 23:31:22,285	INFO services.py:809 -- Starting Redis shard with 10.0 GB max memory.
2019-08-24 23:31:22,423	INFO node.py:512 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-08-24_23-31-21_722734_101587/logs.
2019-08-24 23:31:22,428	WARNING services.py:1301 -- Warning: Capping object memory store to 20.0GB. To increase this further, specify `object_store_memory` when calling ray.init() or ray start.
2019-08-24 23:31:22,430	WARNING services.py:1330 -- WARNING: The default object store size of 20.0 GB will use more than 50% of the available memory on this node (32.01 GB). Consider setting the objec

{'node_ip_address': '10.0.5.4',
 'redis_address': '10.0.5.4:16309',
 'object_store_address': '/tmp/ray/session_2019-08-24_23-31-21_722734_101587/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2019-08-24_23-31-21_722734_101587/sockets/raylet',
 'webui_url': None,
 'session_dir': '/tmp/ray/session_2019-08-24_23-31-21_722734_101587'}

# Create a custom RNN policy

In [4]:
import ray
from ray import tune
from ray.tune.registry import register_env
from ray.rllib.models import ModelCatalog
from ray.rllib.models.modelv2 import ModelV2
from ray.rllib.models.tf.recurrent_tf_modelv2 import RecurrentTFModelV2
from ray.rllib.utils.annotations import override
from ray.rllib.utils import try_import_tf
import tensorflow as tf

In [5]:
class MyKerasRNN(RecurrentTFModelV2):
    """Example of using the Keras functional API to define a RNN model."""

    def __init__(self,
                 obs_space,
                 action_space,
                 num_outputs,
                 model_config,
                 name,
                 hiddens_size=256,
                 cell_size=64):
        super(MyKerasRNN, self).__init__(obs_space, action_space, num_outputs,
                                         model_config, name)
        self.cell_size = cell_size

        # Define input layers
        input_layer = tf.keras.layers.Input(
            shape=(None, obs_space.shape[0]), name="inputs")
        state_in_h = tf.keras.layers.Input(shape=(cell_size, ), name="h")
        state_in_c = tf.keras.layers.Input(shape=(cell_size, ), name="c")
        seq_in = tf.keras.layers.Input(shape=(), name="seq_in")

        # Preprocess observation with a hidden layer and send to LSTM cell
        dense1 = tf.keras.layers.Dense(
            hiddens_size, activation=tf.nn.relu, name="dense1")(input_layer)
        lstm_out, state_h, state_c = tf.keras.layers.LSTM(
            cell_size, return_sequences=True, return_state=True, name="lstm")(
                inputs=dense1,
                mask=tf.sequence_mask(seq_in),
                initial_state=[state_in_h, state_in_c])

        # Postprocess LSTM output with another hidden layer and compute values
        logits = tf.keras.layers.Dense(
            self.num_outputs,
            activation=tf.keras.activations.linear,
            name="logits")(lstm_out)
        values = tf.keras.layers.Dense(
            1, activation=None, name="values")(lstm_out)

        # Create the RNN model
        self.rnn_model = tf.keras.Model(
            inputs=[input_layer, seq_in, state_in_h, state_in_c],
            outputs=[logits, values, state_h, state_c])
        self.register_variables(self.rnn_model.variables)
        self.rnn_model.summary()

    @override(RecurrentTFModelV2)
    def forward_rnn(self, inputs, state, seq_lens):
        model_out, self._value_out, h, c = self.rnn_model([inputs, seq_lens] +
                                                          state)
        return model_out, [h, c]

    @override(ModelV2)
    def get_initial_state(self):
        return [
            np.zeros(self.cell_size, np.float32),
            np.zeros(self.cell_size, np.float32),
        ]

    @override(ModelV2)
    def value_function(self):
        return tf.reshape(self._value_out, [-1])

In [6]:
ModelCatalog.register_custom_model("rnn", MyKerasRNN)

## Create the walk-forward partitions
It's responsibility of the user to create the training/test (and maybe validation) partitions to turn the walk forward training. Note that 2-fold split is a particular case of walk-forward training, so you are still able to run a simple 2-fold split.


In [7]:
partitions = list()
for year in range(2007, 2018):
    partition = {
        'training-set': [datetime.min, datetime(year, 12, 31)],
        'test-set': [datetime(year + 1, 1, 1), datetime(year + 1, 12, 31)],
    }
    partitions.append(partition)
partitions


[{'training-set': [datetime.datetime(1, 1, 1, 0, 0),
   datetime.datetime(2007, 12, 31, 0, 0)],
  'test-set': [datetime.datetime(2008, 1, 1, 0, 0),
   datetime.datetime(2008, 12, 31, 0, 0)]},
 {'training-set': [datetime.datetime(1, 1, 1, 0, 0),
   datetime.datetime(2008, 12, 31, 0, 0)],
  'test-set': [datetime.datetime(2009, 1, 1, 0, 0),
   datetime.datetime(2009, 12, 31, 0, 0)]},
 {'training-set': [datetime.datetime(1, 1, 1, 0, 0),
   datetime.datetime(2009, 12, 31, 0, 0)],
  'test-set': [datetime.datetime(2010, 1, 1, 0, 0),
   datetime.datetime(2010, 12, 31, 0, 0)]},
 {'training-set': [datetime.datetime(1, 1, 1, 0, 0),
   datetime.datetime(2010, 12, 31, 0, 0)],
  'test-set': [datetime.datetime(2011, 1, 1, 0, 0),
   datetime.datetime(2011, 12, 31, 0, 0)]},
 {'training-set': [datetime.datetime(1, 1, 1, 0, 0),
   datetime.datetime(2011, 12, 31, 0, 0)],
  'test-set': [datetime.datetime(2012, 1, 1, 0, 0),
   datetime.datetime(2012, 12, 31, 0, 0)]},
 {'training-set': [datetime.datetime(1, 

In [8]:
## Create the config dict
config = ray.rllib.agents.ppo.DEFAULT_CONFIG.copy()
config['env'] = GAIAPredictorsContinuousV13
config['env_config'] = {
    'cost_of_commissions': tune.grid_search([0.00005]),
    'cost_of_spread': 0.0001,
}
config['gamma'] = tune.grid_search([0.82]) 

In [9]:
config['vf_clip_param'] = 0
config['vf_loss_coeff'] = 1e-05
config['lambda'] = 0
config['use_gae'] = True

# need to have vf share layers if lstm is used
# config['vf_share_layers'] = True
# config['model']['use_lstm'] = True


# config['model']['custom_model'] = 'rnn'
# config['model']['custom_model']['max_seq_len'] = 20

config['batch_mode'] = 'complete_episodes'
config['train_batch_size'] = 4000 # tune.grid_search([4000])
config['sgd_minibatch_size'] = 128
config['num_sgd_iter'] = tune.grid_search([8])
config['entropy_coeff'] =  1e-5 # tune.grid_search([1e-5])
config['kl_coeff'] = 0.2 #tune.grid_search([0.2])
config['kl_target'] = tune.grid_search([0.01])
config['clip_param'] = tune.grid_search([0.8])

config['lr'] = tune.grid_search([1e-5])

# config['model']['custom_model'] = CNN.__name__


In [10]:
# print(config)

In [11]:
# Linear regression
# config['model']['fcnet_hiddens'] = []

In [12]:
# TODO: Use a linear regression.

In [13]:
env = GAIAPredictorsContinuousV13()
env.action_space.sample()

array([-0.05575708, -0.07833881])

In [14]:
## Run your walk-forward experiment
walk_forward = WalkForwardRunner(
    env_partitions=partitions,
    trainable=ray.rllib.agents.ppo.PPOTrainer,
    config=config,
    stop={'timesteps_total': 500000},
    checkpoint_freq=1,
)

Note that WalkForwardRunner has constructed the implied ray Experiment(s) from your walk forward settings.

walk_forward.experiments

Note that trials are associated with a `RestoreID`. This `ID` is all you need to restore an agent. Here we are using a grid search of two values for `cost_of_commissions` on two partitions, so we have a total of 4 experiments.

In [15]:
trials = walk_forward.run(verbose=0)
trials

2019-08-24 00:05:20,169	WARNING experiment.py:223 -- All experiments will be using the same SearchAlgorithm.
2019-08-24 00:05:20,180	INFO trial_runner.py:176 -- Starting a new experiment.
2019-08-24 00:05:20,224	WARNING signature.py:108 -- The function with_updates has a **kwargs argument, which is currently not supported.
2019-08-24 00:05:20,261	ERROR log_sync.py:34 -- Log sync requires cluster to be setup with `ray up`.
2019-08-24 00:05:20,337	WARNING util.py:145 -- The `start_trial` operation took 0.1316242218017578 seconds to complete, which may be a performance bottleneck.


(pid=81041) /home/Nicholas/trading-gym_0.8.1/venv0.8.1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=81041)   _np_qint8 = np.dtype([("qint8", np.int8, 1)])
(pid=81041) /home/Nicholas/trading-gym_0.8.1/venv0.8.1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=81041)   _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
(pid=81041) /home/Nicholas/trading-gym_0.8.1/venv0.8.1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=81041)  

(pid=81038) 2019-08-24 00:05:52,120	INFO rollout_worker.py:319 -- Creating policy evaluation worker 2 on CPU (please ignore any CUDA init errors)
(pid=81038) 2019-08-24 00:05:52.176183: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
(pid=81038) /home/Nicholas/trading-gym_0.8.1/venv0.8.1/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning:
(pid=81038) 
(pid=81038) Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
(pid=81038) 
(pid=81034) 2019-08-24 00:05:55,490	INFO rollout_worker.py:319 -- Creating policy evaluation worker 1 on CPU (please ignore any CUDA init errors)
(pid=81034) 2019-08-24 00:05:55.536508: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
(pid=81041) 2019-08-24 00:05:56,527	INFO train

(pid=81040) /home/Nicholas/trading-gym_0.8.1/venv0.8.1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=81040)   _np_qint8 = np.dtype([("qint8", np.int8, 1)])
(pid=81040) /home/Nicholas/trading-gym_0.8.1/venv0.8.1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=81040)   _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
(pid=81040) /home/Nicholas/trading-gym_0.8.1/venv0.8.1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=81040)  

(pid=81034) 2019-08-24 00:06:03,699	INFO rollout_worker.py:485 -- Completed sample batch:
(pid=81034) 
(pid=81034) { 'data': { 'action_prob': np.ndarray((205,), dtype=float32, min=0.0, max=0.157, mean=0.081),
(pid=81034)             'actions': np.ndarray((205, 2), dtype=float32, min=-3.541, max=2.845, mean=-0.003),
(pid=81034)             'advantages': np.ndarray((205,), dtype=float32, min=-0.009, max=0.014, mean=0.0),
(pid=81034)             'agent_index': np.ndarray((205,), dtype=int64, min=0.0, max=0.0, mean=0.0),
(pid=81034)             'behaviour_logits': np.ndarray((205, 4), dtype=float32, min=-0.009, max=0.013, mean=0.0),
(pid=81034)             'dones': np.ndarray((205,), dtype=bool, min=0.0, max=1.0, mean=0.024),
(pid=81034)             'eps_id': np.ndarray((205,), dtype=int64, min=500771995.0, max=1269115279.0, mean=831006784.0),
(pid=81034)             'infos': np.ndarray((205,), dtype=object, head={'time': Timestamp('2006-07-28 00:00:00'), 'nlv': 99.99863640908863, 'nr_cont

...

(pid=114117) 2019-08-24 12:34:28,740	WARNING ppo.py:121 -- The magnitude of your environment rewards are more than infx the scale of `vf_clip_param`. This means that it will take more than inf iterations for your value function to converge. If this is not intended, consider increasing `vf_clip_param`.


2019-08-24 12:34:33,913	WARNING util.py:145 -- The `process_trial` operation took 0.1101837158203125 seconds to complete, which may be a performance bottleneck.


(pid=114117) 2019-08-24 12:34:54,719	WARNING ppo.py:121 -- The magnitude of your environment rewards are more than infx the scale of `vf_clip_param`. This means that it will take more than inf iterations for your value function to converge. If this is not intended, consider increasing `vf_clip_param`.


2019-08-24 12:34:59,073	WARNING util.py:145 -- The `process_trial` operation took 0.16121292114257812 seconds to complete, which may be a performance bottleneck.


[PPO_GAIAPredictorsContinuousV13_restoreID=4420155622729557137_runID=Rb4JG32f_0_clip_param=0.8,cost_of_commissions=5e-05,gamma=0.82,kl_target=0.01,lr=1e-05,num_sgd_iter=8,
 PPO_GAIAPredictorsContinuousV13_restoreID=4420155622729557137_runID=Rb4JG32f_0_clip_param=0.8,cost_of_commissions=5e-05,gamma=0.82,kl_target=0.01,lr=1e-05,num_sgd_iter=8,
 PPO_GAIAPredictorsContinuousV13_restoreID=4420155622729557137_runID=Rb4JG32f_0_clip_param=0.8,cost_of_commissions=5e-05,gamma=0.82,kl_target=0.01,lr=1e-05,num_sgd_iter=8,
 PPO_GAIAPredictorsContinuousV13_restoreID=4420155622729557137_runID=Rb4JG32f_0_clip_param=0.8,cost_of_commissions=5e-05,gamma=0.82,kl_target=0.01,lr=1e-05,num_sgd_iter=8,
 PPO_GAIAPredictorsContinuousV13_restoreID=4420155622729557137_runID=Rb4JG32f_0_clip_param=0.8,cost_of_commissions=5e-05,gamma=0.82,kl_target=0.01,lr=1e-05,num_sgd_iter=8,
 PPO_GAIAPredictorsContinuousV13_restoreID=4420155622729557137_runID=Rb4JG32f_0_clip_param=0.8,cost_of_commissions=5e-05,gamma=0.82,kl_targe

# Restore

In [ ]:
# need and want to restoere the most recent one that has gamma = 0.82

In [5]:
results = WalkForwardResults('/home/Nicholas/trading-gym_0.8.1/trading-gym/notebooks/registry/gaia/v13/logs')
results

WalkForwardResults(['GAIAPredictorsContinuousV13'])

In [6]:
env_results = results['GAIAPredictorsContinuousV13']
env_results

EnvResults(GAIAPredictorsContinuousV13)

In [7]:
# Step 1.
env = env_results.make_env(
    env_config={
        'cost_of_commissions': 0.00005,
        'cost_of_spread': 0.0001,
        'folds': {
            'training-set': [datetime.min, datetime(2008, 3, 18)],
            'test-set': [datetime(2008, 3, 19), datetime.max],
        }
    },
)

**NOTE:**<br>
3892154612095477129 is for linear regression<br>
-7538808210974229658 is for non-linear, basic MLP <br>
-330274535200891787 for non-linear, basic MLP with LSTM

In [8]:
env_results.restore_ids

{4420155622729557137: [AgentResults(GAIAPredictorsContinuousV13_1-01-01_to_2012-12-31/PPO_GAIAPredictorsContinuousV13_restoreID=4420155622729557137_runID=Rb4JG32f_0_clip_param=0.8,cost_of_commissions=5e-05,gamma=0.82_2019-08-24_05-51-09hb9oeay0),
  AgentResults(GAIAPredictorsContinuousV13_1-01-01_to_2007-12-31/PPO_GAIAPredictorsContinuousV13_restoreID=4420155622729557137_runID=Rb4JG32f_0_clip_param=0.8,cost_of_commissions=5e-05,gamma=0.82_2019-08-24_00-05-2082flg21m),
  AgentResults(GAIAPredictorsContinuousV13_1-01-01_to_2014-12-31/PPO_GAIAPredictorsContinuousV13_restoreID=4420155622729557137_runID=Rb4JG32f_0_clip_param=0.8,cost_of_commissions=5e-05,gamma=0.82_2019-08-24_08-10-31nfxzgs19),
  AgentResults(GAIAPredictorsContinuousV13_1-01-01_to_2008-12-31/PPO_GAIAPredictorsContinuousV13_restoreID=4420155622729557137_runID=Rb4JG32f_0_clip_param=0.8,cost_of_commissions=5e-05,gamma=0.82_2019-08-24_01-17-35bj94l_1z),
  AgentResults(GAIAPredictorsContinuousV13_1-01-01_to_2015-12-31/PPO_GAIAPr

In [9]:
env_results_nonlinear = env_results.get_nr2episode(
    restore_id=4420155622729557137,
    checkpoint_nrs=np.arange(1, 100, 1),
    fold='test-set',
    env_config={
        'folds': {
            'training-set': [datetime.min, datetime(2008, 3, 18)],
            'test-set': [datetime(2008, 3, 19), datetime.max],
        }
    }
)
# env_results_linear = env_results.get_nr2episode(
#     restore_id=3892154612095477129,
#     checkpoint_nrs=np.arange(1, 126, 1),
#     fold='test-set',
#     env_config={
#         'folds': {
#             'training-set': [datetime.min, datetime(2008, 3, 18)],
#             'test-set': [datetime(2008, 3, 19), datetime.max],
#         }
#     }
# )

2019-08-24 23:31:37,223	ERROR log_sync.py:34 -- Log sync requires cluster to be setup with `ray up`.
2019-08-24 23:31:37,261	WARNING ppo.py:143 -- FYI: By default, the value function will not share layers with the policy model ('vf_share_layers': False).
2019-08-24 23:31:39,249	INFO rollout_worker.py:319 -- Creating policy evaluation worker 0 on CPU (please ignore any CUDA init errors)
2019-08-24 23:31:39,903	INFO dynamic_tf_policy.py:324 -- Initializing loss function with dummy input:

{ 'action_prob': <tf.Tensor 'default_policy/action_prob:0' shape=(?,) dtype=float32>,
  'actions': <tf.Tensor 'default_policy/actions:0' shape=(?, 2) dtype=float32>,
  'advantages': <tf.Tensor 'default_policy/advantages:0' shape=(?,) dtype=float32>,
  'behaviour_logits': <tf.Tensor 'default_policy/behaviour_logits:0' shape=(?, 4) dtype=float32>,
  'dones': <tf.Tensor 'default_policy/dones:0' shape=(?,) dtype=bool>,
  'new_obs': <tf.Tensor 'default_policy/new_obs:0' shape=(?, 23) dtype=float32>,
  'obs':

(pid=101984) /home/Nicholas/trading-gym_0.8.1/venv0.8.1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=101984)   _np_qint8 = np.dtype([("qint8", np.int8, 1)])
(pid=101984) /home/Nicholas/trading-gym_0.8.1/venv0.8.1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=101984)   _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
(pid=101984) /home/Nicholas/trading-gym_0.8.1/venv0.8.1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=101

2019-08-24 23:31:59,525	INFO trainable.py:105 -- _setup took 22.298 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2019-08-24 23:31:59,529	WARNING util.py:47 -- Install gputil for GPU system monitoring.


(pid=101986) /home/Nicholas/trading-gym_0.8.1/venv0.8.1/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning:
(pid=101986) 
(pid=101986) Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
(pid=101986) 


2019-08-24 23:31:59,795	INFO tf_run_builder.py:92 -- Executing TF run without tracing. To dump TF timeline traces to disk, set the TF_TIMELINE_DIR environment variable.
2019-08-24 23:32:02,204	WARNING ppo.py:143 -- FYI: By default, the value function will not share layers with the policy model ('vf_share_layers': False).
2019-08-24 23:32:04,627	INFO rollout_worker.py:319 -- Creating policy evaluation worker 0 on CPU (please ignore any CUDA init errors)
2019-08-24 23:32:08,596	INFO rollout_worker.py:742 -- Built policy map: {'default_policy': <ray.rllib.policy.tf_policy_template.PPOTFPolicy object at 0x7fcc74ddefd0>}
2019-08-24 23:32:08,599	INFO rollout_worker.py:743 -- Built preprocessor map: {'default_policy': <ray.rllib.models.preprocessors.NoPreprocessor object at 0x7fcc74ddeba8>}
2019-08-24 23:32:08,600	INFO rollout_worker.py:356 -- Built filter map: {'default_policy': <ray.rllib.utils.filter.NoFilter object at 0x7fcc75ab9b00>}
2019-08-24 23:32:08,727	INFO multi_gpu_optimizer.py:93

...

(pid=101991) /home/Nicholas/trading-gym_0.8.1/venv0.8.1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=101991)   _np_qint8 = np.dtype([("qint8", np.int8, 1)])
(pid=101991) /home/Nicholas/trading-gym_0.8.1/venv0.8.1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=101991)   _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
(pid=101991) /home/Nicholas/trading-gym_0.8.1/venv0.8.1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=101

2019-08-24 23:32:40,199	INFO trainable.py:105 -- _setup took 16.884 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2019-08-24 23:32:40,201	WARNING util.py:47 -- Install gputil for GPU system monitoring.
2019-08-24 23:32:42,394	WARNING ppo.py:143 -- FYI: By default, the value function will not share layers with the policy model ('vf_share_layers': False).
2019-08-24 23:32:44,344	INFO rollout_worker.py:319 -- Creating policy evaluation worker 0 on CPU (please ignore any CUDA init errors)
2019-08-24 23:32:47,805	INFO rollout_worker.py:742 -- Built policy map: {'default_policy': <ray.rllib.policy.tf_policy_template.PPOTFPolicy object at 0x7fc776fa9be0>}
2019-08-24 23:32:47,808	INFO rollout_worker.py:743 -- Built preprocessor map: {'default_policy': <ray.rllib.models.preprocessors.NoPreprocessor object at 0x7fc776fa97b8>}
2019-08-24 23:32:47,815	INFO rollout_worker.py:356 -- Built filter map: {'default_policy': <ray.r

(pid=101988) /home/Nicholas/trading-gym_0.8.1/venv0.8.1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=101988)   _np_qint8 = np.dtype([("qint8", np.int8, 1)])
(pid=101988) /home/Nicholas/trading-gym_0.8.1/venv0.8.1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=101988)   _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
(pid=101988) /home/Nicholas/trading-gym_0.8.1/venv0.8.1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=101

2019-08-24 23:32:59,132	INFO trainable.py:105 -- _setup took 16.741 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2019-08-24 23:32:59,134	WARNING util.py:47 -- Install gputil for GPU system monitoring.
2019-08-24 23:33:01,826	WARNING ppo.py:143 -- FYI: By default, the value function will not share layers with the policy model ('vf_share_layers': False).
2019-08-24 23:33:04,131	INFO rollout_worker.py:319 -- Creating policy evaluation worker 0 on CPU (please ignore any CUDA init errors)
2019-08-24 23:33:08,283	INFO rollout_worker.py:742 -- Built policy map: {'default_policy': <ray.rllib.policy.tf_policy_template.PPOTFPolicy object at 0x7fc7661699b0>}
2019-08-24 23:33:08,285	INFO rollout_worker.py:743 -- Built preprocessor map: {'default_policy': <ray.rllib.models.preprocessors.NoPreprocessor object at 0x7fc766169588>}
2019-08-24 23:33:08,286	INFO rollout_worker.py:356 -- Built filter map: {'default_policy': <ray.r

2019-08-24 23:34:59,770	WARNING util.py:47 -- Install gputil for GPU system monitoring.


## Weights

### Non-linear

In [10]:
env_results_nonlinear.plot_weights()

interactive(children=(IntSlider(value=50, description='nr', max=150, min=-50), Output()), _dom_classes=('widge…

FigureWidget({
    'data': [{'name': 'Cash(USD)',
              'type': 'scatter',
              'uid': '4f409…

### Linear

In [11]:
# env_results_linear.plot_weights()

## Levels

### Non-linear

In [12]:
env_results_nonlinear.plot_levels() 

interactive(children=(IntSlider(value=50, description='nr', max=150, min=-50), Output()), _dom_classes=('widge…

FigureWidget({
    'data': [{'name': 'Strategy',
              'type': 'scatter',
              'uid': 'a68c77…

### Linear

In [13]:
# env_results_linear.plot_levels()

## Metrics

### Non-linear

In [14]:
env_results_nonlinear.plot_metrics_as_we_train()

### Linear

In [15]:
# env_results_linear.plot_metrics_as_we_train()

In [16]:
restore_id = 4420155622729557137

In [17]:
policy = env_results.make_policy(
    env=env,
    restore_id=restore_id,
    checkpoint_nr=None,  # use None (or don't specify) to use last checkpoint available
)
policy

In [18]:
# Step 3.
episode = env.sample_episode(fold='test-set', policy=policy, verbose=False)

In [19]:
episode.renderer.cumulative_performance.to_plotly()
episode.renderer.target_weights.to_plotly()
episode.renderer.annual_returns.to_plotly()
episode.renderer.tearsheet()

Strategy  \
Context              From                              2008-03-19   
                     To                                2018-08-28   
                     Years                                10.4493   
                     Observations                            2725   
                     Risk-free asset        Index(USD 1M Deposit)   
                     Risk-free CAGR                    0.00681294   
Return               CAGR                                0.104083   
                     CAGR over cash                     0.0972701   
                     Overall return                       1.81409   
Risk                 Volatility                          0.105432   
                     Downside volatility                  0.07812   
                     Upside volatility                  0.0752399   
                     Max drawdown                       -0.160244   
                     Martin risk                        0.0359461   
                     VaR 5%                            -0.0103589   
                     VaR 2%                            -0.0148512   
                     Expected shortfall 5%             -0.0157475   
                     Expected shortfall 2%             -0.0206279   
Risk-adjusted return Sharpe ratio                        0.922588   
                     Sortino ratio                        1.24514   
                     Calmar ratio                        0.607012   
                     Martin ratio                           2.706   
Outperformance       Benchmark name         Index(Aric-Benchmark)   
                     CAGR over benchmark               -0.0545026   
                     Information ratio                  -0.534719   
                     CAPM Alpha                         0.0192595   
                     CAPM Beta                           0.538523   
Weights              Cash(USD)                         0.00522083   
                     ETF(Russell 1000)                   0.518945   
                     ETF(7-10Y T-Bills)                  0.475834   
                     Leverage mean                              1   
                     Turnover daily                     0.0644596   
                     Turnover annual                      16.2438   

                                            Index(Aric-Benchmark)  \
Context              From                              2008-03-19   
                     To                                2018-08-28   
                     Years                                10.4493   
                     Observations                            2725   
                     Risk-free asset        Index(USD 1M Deposit)   
                     Risk-free CAGR                    0.00681294   
Return               CAGR                                0.158586   
                     CAGR over cash                      0.151773   
                     Overall return                       3.65592   
Risk                 Volatility                         0.0970738   
                     Downside volatility                 0.069906   
                     Upside volatility                  0.0705399   
                     Max drawdown                      -0.0865477   
                     Martin risk                        0.0189993   
                     VaR 5%                           -0.00895352   
                     VaR 2%                            -0.0134313   
                     Expected shortfall 5%             -0.0137858   
                     Expected shortfall 2%              -0.017882   
Risk-adjusted return Sharpe ratio                         1.56348   
                     Sortino ratio                         2.1711   
                     Calmar ratio                         1.75363   
                     Martin ratio                         7.98834   
Outperformance       Benchmark name         Index(Aric-Benchmark)   
                     CAGR over benchmark              